<img style="float: right;" src="meetup_logo.svg" width=200>

# Meetup - Data Cleaning


<i>Cleaning the contents of the NYC meetup data</i>

<u>Datasets:</u>

1. <a href='#events'>Meetup Events</a> (from Sept. 1, 2018 - Sept. 31, 2018)
2. <a href='#groups'>Meetup Groups</a>
3. <a href='#members'>Meetup Members</a>
***

### Import libraries

In [142]:
from haversine import haversine 
import reverse_geocode
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
from meetup_api_functions import clean_text
from meetup_api_functions import get_subway_distances
import requests

***
<a id='events'></a>
### 1. Meetup Events

#### Load Data

In [146]:
# open all_events file
with open('all_past_events.pkl', 'rb') as f:
    all_past_events = pickle.load(f)

In [147]:
print("There were {} events in September 2018 in NYC.".format(len(all_past_events)))

There were 12866 events in September 2018 in NYC.


In [148]:
# convert all_events into a dataframe
df_events = pd.DataFrame(all_past_events)

In [149]:
# preview the information
df_events.head()

,created,description,duration,event_url,fee,group,headcount,how_to_find_us,id,maybe_rsvp_count,...,rsvp_limit,status,time,updated,utc_offset,venue,visibility,waitlist_count,why,yes_rsvp_count
0,1535391367000,<p>Join us in person or tune in online!</p> <p...,7200000.0,https://www.meetup.com/Build-with-Code-New-Yor...,NaN,"{'join_mode': 'open', 'created': 1484876702000...",0,Please come to second floor/ stream online at ...,254149786,0,...,NaN,past,1536100200000,1536109876000,-14400000,"{'country': 'us', 'localized_country_name': 'U...",public,0,NaN,42
1,1535385547000,<p>Get started now on challenges related to th...,9000000.0,https://www.meetup.com/Build-with-Code-New-Yor...,NaN,"{'join_mode': 'open', 'created': 1484876702000...",0,https://zoom.us/j/417883916,254146381,0,...,NaN,past,1536192000000,1536203991000,-14400000,"{'country': 'us', 'localized_country_name': 'U...",public,0,NaN,64
2,1535392484000,<p>In this workshop we’ll get a clear sense of...,9000000.0,https://www.meetup.com/Build-with-Code-New-Yor...,NaN,"{'join_mode': 'open', 'created': 1484876702000...",0,Please come to second floor / Livestream at ht...,254150230,0,...,NaN,past,1536273000000,1536287034000,-14400000,"{'country': 'us', 'localized_country_name': 'U...",public,0,NaN,83
3,1531947994000,<p>The number of opportunities for software en...,7200000.0,https://www.meetup.com/Build-with-Code-New-Yor...,NaN,"{'join_mode': 'open', 'created': 1484876702000...",0,Please come to the 2nd floor,252915161,0,...,NaN,past,1536359400000,1536370996000,-14400000,"{'country': 'us', 'localized_country_name': 'U...",public,0,NaN,113
4,1535383458000,"<p>Please tune into the stream here: <a href=""...",7200000.0,https://www.meetup.com/Build-with-Code-New-Yor...,NaN,"{'join_mode': 'open', 'created': 1484876702000...",0,Please tune into the stream here: https://zoom...,254144933,0,...,NaN,past,1536629400000,1536639324000,-14400000,"{'country': 'us', 'localized_country_name': 'U...",public,0,NaN,21


#### Data Cleaning

In [150]:
# check percentage of NaN values in each column
df_events.isna().sum()/len(df_events)*100

created              0.000000
description          0.645111
duration             5.160889
event_url            0.000000
fee                 85.714286
group                0.000000
headcount            0.000000
how_to_find_us      61.153428
id                   0.000000
maybe_rsvp_count     0.000000
name                 0.000000
photo_url           39.709311
rating               0.000000
rsvp_limit          74.436499
status               0.000000
time                 0.000000
updated              0.000000
utc_offset           0.000000
venue                8.215452
visibility           0.000000
waitlist_count       0.000000
why                 99.556972
yes_rsvp_count       0.000000
dtype: float64

Based on the information above, there is some data cleaning and handling of missing values to address:
- convert relevants columns to datetime format (```created```, ```time```, ```updated```)
- convert values in ```duration``` from miliseconds to minutes and fill in the missing values with median
- drop ```utc_offset``` since that information is captured in ```time```
- drop ```why``` since most values are NaNs
- drop ```how_to_find_us``` since almost half are missing values and this detail is not crucial to analysis
- clean up text in ```description```
- ```fee```
- ```venue``` fill missing values with 'None'
-```photo_url```

In [151]:
# convert timestamps to human-readable format (dividing by 1000. since time is in milliseconds)
df_events['created'] = df_events['created'].apply(lambda x:time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(x/1000.)))
df_events['time'] = df_events['time'].apply(lambda x:time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(x/1000.)))
df_events['updated'] = df_events['updated'].apply(lambda x:time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(x/1000.)))

In [152]:
# convert values in duration column from milliseconds to minutes
df_events['duration'] = df_events['duration'].apply(lambda x: x/60000)

In [153]:
# rename column to note time unit of the data
df_events.rename(columns={'duration':'duration_min'}, inplace=True)

In [154]:
# dropping unnecessary columns
df_events.drop(columns = ['why', 'utc_offset', 'how_to_find_us'], inplace = True)

In [155]:
# clean text in description using regex
df_events.description[0]

'<p>Join us in person or tune in online!</p> <p>Livestream: <a href="https://zoom.us/j/190996928" class="linkified">https://zoom.us/j/190996928</a></p> <p>Get started now on challenges related to these topics on our FREE online learning platform CSX, <a href="https://csx.codesmith.io/" class="linkified">https://csx.codesmith.io/</a></p> <p>--</p> <p>During this workshop, we will cover:</p> <p>- What happens when our code runs in the browser?<br/>- A closer look at objects<br/>- Reusing our logic (declaring/invoking functions)</p> <p>These concepts are the foundation of all web development - we will cover them under-the-hood so you can confidently use them as you work on harder concepts to come.</p> <p>Schedule</p> <p>6:30 - 7:00pm: Meet your future pair programming partner.</p> <p>7:00 - 8:00pm: Core JavaScript concept for the challenge and introduction to the secret hack for learning to code - pair-programming.</p> <p>8:00 - 9:00pm: Pair-programming.</p> <p>***Bring a friend who\'d li

In [156]:
# but first replace NaN in description column with 'None'
df_events.description.fillna(value = 'None', inplace = True)

In [157]:
# function to clean text in the description column
def clean_text(description):
    """return cleaned text; requires a string"""
    result1 = re.sub('&lt;br/&gt;', '', re.sub('<[^>]+>', '', description))
    final = re.sub('&lt;/p&gt;', '', re.sub('&amp;', "", result1))
    return final

In [158]:
# apply/lambda to clean all data in the description series
df_events['description'] = df_events['description'].apply(lambda x: clean_text(x))

In [159]:
# cleaned text in description
df_events.description[0]

"Join us in person or tune in online! Livestream: https://zoom.us/j/190996928 Get started now on challenges related to these topics on our FREE online learning platform CSX, https://csx.codesmith.io/ -- During this workshop, we will cover: - What happens when our code runs in the browser?- A closer look at objects- Reusing our logic (declaring/invoking functions) These concepts are the foundation of all web development - we will cover them under-the-hood so you can confidently use them as you work on harder concepts to come. Schedule 6:30 - 7:00pm: Meet your future pair programming partner. 7:00 - 8:00pm: Core JavaScript concept for the challenge and introduction to the secret hack for learning to code - pair-programming. 8:00 - 9:00pm: Pair-programming. ***Bring a friend who'd like to build and you can pair program together! Price: Always free For those online! Please join the stream here: Livestream: https://zoom.us/j/190996928"

In [160]:
# summary statistics of event duration (in minutes)
df_events.duration_min.describe()

count    12202.000000
mean       305.118341
std       1174.125784
min          1.000000
25%        120.000000
50%        135.000000
75%        210.000000
max      18840.000000
Name: duration_min, dtype: float64

In [161]:
# look at sample non-NaN value in fee column
df_events.fee[12838]

{'amount': 10,
 'accepts': 'cash',
 'description': 'per person',
 'currency': 'USD',
 'label': 'Price',
 'required': '0'}

In [162]:
# replace missing values in duration to median value
# replace missing venue values with 'None'
# replace missing fee values with 'N/A'
# replace missing rsvp_limit values with 'N/A'
df_events.duration_min.fillna(value = df_events.duration_min.median(), inplace = True)
df_events.venue.fillna(value = 'None', inplace = True)
df_events.fee.fillna(value = 0, inplace = True)
df_events.rsvp_limit.fillna(value = 'N/A', inplace = True)

In [163]:
# extract just the amount from the fee dictionary
df_events.fee = df_events.fee.apply(lambda x: x['amount'] if x!= 0 else 0)

In [164]:
# preview the cleaned dataframe
df_events.head()

,created,description,duration_min,event_url,fee,group,headcount,id,maybe_rsvp_count,name,photo_url,rating,rsvp_limit,status,time,updated,venue,visibility,waitlist_count,yes_rsvp_count
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254149786,0,"JavaScript The Easier Parts: Objects, Function...",NaN,"{'count': 0, 'average': 0}",N/A,past,09/04/2018 22:30:00,09/05/2018 01:11:16,"{'country': 'us', 'localized_country_name': 'U...",public,0,42
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,NaN,"{'count': 0, 'average': 0}",N/A,past,09/06/2018 00:00:00,09/06/2018 03:19:51,"{'country': 'us', 'localized_country_name': 'U...",public,0,64
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254150230,0,JavaScript The Hard Parts - Promises and Async...,NaN,"{'count': 0, 'average': 0}",N/A,past,09/06/2018 22:30:00,09/07/2018 02:23:54,"{'country': 'us', 'localized_country_name': 'U...",public,0,83
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,252915161,0,How to get Hired as a Software Engineer,NaN,"{'count': 0, 'average': 0}",N/A,past,09/07/2018 22:30:00,09/08/2018 01:43:16,"{'country': 'us', 'localized_country_name': 'U...",public,0,113
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254144933,0,Codesmith Online Info Session (Online Only),NaN,"{'count': 0, 'average': 0}",N/A,past,09/11/2018 01:30:00,09/11/2018 04:15:24,"{'country': 'us', 'localized_country_name': 'U...",public,0,21


In [165]:
# there are no more NaN values in df_events
df_events.isna().sum()

created                0
description            0
duration_min           0
event_url              0
fee                    0
group                  0
headcount              0
id                     0
maybe_rsvp_count       0
name                   0
photo_url           5109
rating                 0
rsvp_limit             0
status                 0
time                   0
updated                0
venue                  0
visibility             0
waitlist_count         0
yes_rsvp_count         0
dtype: int64

Now let's look at the ```venue``` column.

It is a dictionary itself so we can change that series into its own dataframe.

In [166]:
# converting the 'venue' column into its own dataframe
df_venues = df_events['venue'].apply(pd.Series)
df_venues.head()

/Users/gracejeong/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
/Users/gracejeong/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,address_1,city,country,id,lat,localized_country_name,lon,name,phone,repinned,state,zip,0,address_2
0,"250 Lafayette Street, New York, NY",New York,us,25315570.0,40.723171,USA,-73.997177,Codesmith,NaN,False,NY,NaN,NaN,NaN
1,Online,New York,us,25626092.0,40.746730,USA,-73.989670,Online,NaN,True,NY,NaN,NaN,NaN
2,250 Lafayette Street,New York,us,25312065.0,40.723170,USA,-73.997180,Codesmith,NaN,True,NY,NaN,NaN,NaN
3,250 Lafayette Street,New York,us,25312065.0,40.723171,USA,-73.997177,Codesmith,NaN,False,NY,NaN,NaN,NaN
4,Online,New York,us,25626092.0,40.746731,USA,-73.989670,Online,NaN,False,NY,NaN,NaN,NaN


In [167]:
# quick check on the shape of the dataframe to ensure that the number of rows matches the total number of events
df_venues.shape

(12866, 14)

In [168]:
# checking for percentage of missing vaues for each column in the df_event_locations dataframe
(df_venues.isna().sum()/df_venues.shape[0])*100

address_1                  8.215452
city                       8.215452
country                    8.215452
id                         8.215452
lat                        8.215452
localized_country_name     8.215452
lon                        8.215452
name                       8.215452
phone                     92.903777
repinned                   8.215452
state                     19.563190
zip                       64.106949
0                         91.784548
address_2                 94.419400
dtype: float64

We'll need to clean the data here too to address missing values and ensure correct data types.

- replace NaN with ```'None'``` for object data types and ```0``` for integer/float data types
- merge ```lat``` and ```lon``` into a tuple
- drop ```address_2```, ```0```, ```phone```, ```repinned``` since those details are not needed (note: repinned is true if the editor of the event altered the original venues pin location, false otherwise)
- drop ```country``` since we have ```localized_country_name```

In [169]:
# dropping specified columns
df_venues.drop(columns = ['address_2', 0, 'phone', 'country', 'repinned'], inplace = True)
df_venues.head()

,address_1,city,id,lat,localized_country_name,lon,name,state,zip
0,"250 Lafayette Street, New York, NY",New York,25315570.0,40.723171,USA,-73.997177,Codesmith,NY,NaN
1,Online,New York,25626092.0,40.746730,USA,-73.989670,Online,NY,NaN
2,250 Lafayette Street,New York,25312065.0,40.723170,USA,-73.997180,Codesmith,NY,NaN
3,250 Lafayette Street,New York,25312065.0,40.723171,USA,-73.997177,Codesmith,NY,NaN
4,Online,New York,25626092.0,40.746731,USA,-73.989670,Online,NY,NaN


In [170]:
# replace NaN with 'None' for object data types and 0 for integer/float data types
df_venues['address_1'].fillna('None',inplace = True)
df_venues['city'].fillna('None',inplace = True)
df_venues['id'].fillna(0,inplace = True)
df_venues['lat'].fillna(0,inplace = True)
df_venues['localized_country_name'].fillna('None',inplace = True)
df_venues['lon'].fillna(0,inplace = True)
df_venues['name'].fillna('None',inplace = True)
df_venues['state'].fillna('None',inplace = True)
df_venues['zip'].fillna('None',inplace = True)

In [171]:
# merge the lat and lon coordinates into a tuple for reverse geocoding
df_venues['latlon'] = list(zip(df_venues['lat'], df_venues['lon']))
df_venues.head()

,address_1,city,id,lat,localized_country_name,lon,name,state,zip,latlon
0,"250 Lafayette Street, New York, NY",New York,25315570.0,40.723171,USA,-73.997177,Codesmith,NY,None,"(40.723171, -73.997177)"
1,Online,New York,25626092.0,40.746730,USA,-73.989670,Online,NY,None,"(40.74673, -73.98967)"
2,250 Lafayette Street,New York,25312065.0,40.723170,USA,-73.997180,Codesmith,NY,None,"(40.72317, -73.99718)"
3,250 Lafayette Street,New York,25312065.0,40.723171,USA,-73.997177,Codesmith,NY,None,"(40.723171, -73.997177)"
4,Online,New York,25626092.0,40.746731,USA,-73.989670,Online,NY,None,"(40.746731, -73.98967)"


In [172]:
# add neighborhood and country with reverse geocoding
df_venues['neighborhood'] = df_venues['latlon'].apply(lambda x: (reverse_geocode.search([x])[0]['city']) if x[0] != 0 else 'None')
df_venues['geo_country'] = df_venues['latlon'].apply(lambda x: (reverse_geocode.search([x])[0]['country']) if x[0] != 0 else 'None')

In [173]:
# preview updated dataframe
df_venues.head()

,address_1,city,id,lat,localized_country_name,lon,name,state,zip,latlon,neighborhood,geo_country
0,"250 Lafayette Street, New York, NY",New York,25315570.0,40.723171,USA,-73.997177,Codesmith,NY,None,"(40.723171, -73.997177)",Gramercy Park,United States
1,Online,New York,25626092.0,40.746730,USA,-73.989670,Online,NY,None,"(40.74673, -73.98967)",Gramercy Park,United States
2,250 Lafayette Street,New York,25312065.0,40.723170,USA,-73.997180,Codesmith,NY,None,"(40.72317, -73.99718)",Gramercy Park,United States
3,250 Lafayette Street,New York,25312065.0,40.723171,USA,-73.997177,Codesmith,NY,None,"(40.723171, -73.997177)",Gramercy Park,United States
4,Online,New York,25626092.0,40.746731,USA,-73.989670,Online,NY,None,"(40.746731, -73.98967)",Gramercy Park,United States


In [174]:
# add columns to the main df_events dataframe that we want to keep 
df_events['venue_neighborhood'] =  df_venues['neighborhood']
df_events['venue_latlon'] =  df_venues['latlon']
df_events['venue_country'] =  df_venues['geo_country']
df_events['venue_id'] = df_venues['id']

In [175]:
# drop the 'venue' column from df_events
df_events.drop(columns =['venue'], inplace=True)

In [176]:
# preview updated dataframe
df_events.head()

,created,description,duration_min,event_url,fee,group,headcount,id,maybe_rsvp_count,name,...,status,time,updated,visibility,waitlist_count,yes_rsvp_count,venue_neighborhood,venue_latlon,venue_country,venue_id
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254149786,0,"JavaScript The Easier Parts: Objects, Function...",...,past,09/04/2018 22:30:00,09/05/2018 01:11:16,public,0,42,Gramercy Park,"(40.723171, -73.997177)",United States,25315570.0
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,...,past,09/06/2018 00:00:00,09/06/2018 03:19:51,public,0,64,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254150230,0,JavaScript The Hard Parts - Promises and Async...,...,past,09/06/2018 22:30:00,09/07/2018 02:23:54,public,0,83,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,252915161,0,How to get Hired as a Software Engineer,...,past,09/07/2018 22:30:00,09/08/2018 01:43:16,public,0,113,Gramercy Park,"(40.723171, -73.997177)",United States,25312065.0
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,"{'join_mode': 'open', 'created': 1484876702000...",0,254144933,0,Codesmith Online Info Session (Online Only),...,past,09/11/2018 01:30:00,09/11/2018 04:15:24,public,0,21,Gramercy Park,"(40.746731, -73.98967)",United States,25626092.0


Now let's look at the ```group``` column.

It is a dictionary itself so we can change that series into its own dataframe.

In [177]:
# converting the 'group' column into its own dataframe
df_group = df_events['group'].apply(pd.Series)
df_group.head()

,join_mode,created,name,group_lon,id,urlname,group_lat,who
0,open,1484876702000,Build with Code - New York City,-73.989998,21993357,Build-with-Code-New-York,40.75,Engineers
1,open,1484876702000,Build with Code - New York City,-73.989998,21993357,Build-with-Code-New-York,40.75,Engineers
2,open,1484876702000,Build with Code - New York City,-73.989998,21993357,Build-with-Code-New-York,40.75,Engineers
3,open,1484876702000,Build with Code - New York City,-73.989998,21993357,Build-with-Code-New-York,40.75,Engineers
4,open,1484876702000,Build with Code - New York City,-73.989998,21993357,Build-with-Code-New-York,40.75,Engineers


In [178]:
df_group.isna().sum()

join_mode    0
created      0
name         0
group_lon    0
id           0
urlname      0
group_lat    0
who          0
dtype: int64

In [179]:
# add group_id column to df_events and drop the 'group' column
df_events['group_id'] = df_events.group.apply(lambda x: x['id'])
df_events.drop(columns = ['group'], inplace = True)

In [180]:
# rename id column to event_id for clarity
df_events.rename(index=str, columns={"id": "event_id"})

,created,description,duration_min,event_url,fee,headcount,event_id,maybe_rsvp_count,name,photo_url,...,time,updated,visibility,waitlist_count,yes_rsvp_count,venue_neighborhood,venue_latlon,venue_country,venue_id,group_id
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...",NaN,...,09/04/2018 22:30:00,09/05/2018 01:11:16,public,0,42,Gramercy Park,"(40.723171, -73.997177)",United States,25315570.0,21993357
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,NaN,...,09/06/2018 00:00:00,09/06/2018 03:19:51,public,0,64,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0,21993357
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,NaN,...,09/06/2018 22:30:00,09/07/2018 02:23:54,public,0,83,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0,21993357
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,252915161,0,How to get Hired as a Software Engineer,NaN,...,09/07/2018 22:30:00,09/08/2018 01:43:16,public,0,113,Gramercy Park,"(40.723171, -73.997177)",United States,25312065.0,21993357
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254144933,0,Codesmith Online Info Session (Online Only),NaN,...,09/11/2018 01:30:00,09/11/2018 04:15:24,public,0,21,Gramercy Park,"(40.746731, -73.98967)",United States,25626092.0,21993357
5,09/04/2018 17:24:11,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254381681,0,"JavaScript The Easier Parts: Objects, Function...",NaN,...,09/11/2018 22:30:00,09/12/2018 01:10:35,public,0,57,Gramercy Park,"(40.72317, -73.99718)",United States,25315570.0,21993357
6,09/04/2018 16:00:28,Please register for and tune into the stream h...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254379546,0,ONLINE ONLY: Callbacks & Higher Order Functions,NaN,...,09/13/2018 00:00:00,09/13/2018 03:32:41,public,0,63,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0,21993357
7,09/04/2018 20:22:45,"Join us on Thursday, Sep 13, for a great serie...",30.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254387434,0,"Tech Talks: ARKE, SmartGraphQL, Orcha",https://secure.meetupstatic.com/photos/event/9...,...,09/13/2018 22:30:00,09/13/2018 23:47:56,public,0,14,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0,21993357
8,09/04/2018 20:15:41,"Join us on Thursday, for a great series of lig...",150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254387251,0,Tech Talks + JavaScript - The Hard Parts: Clos...,NaN,...,09/13/2018 22:30:00,09/14/2018 02:08:59,public,0,60,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0,21993357
9,08/30/2018 01:17:54,In this online workshop (9am-3pm PST / 12-6pm ...,360.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.000000,0,254226040,0,Build Your Own Web App - 1 Day Online Crash Co...,https://secure.meetupstatic.com/photos/event/b...,...,09/15/2018 16:00:00,09/15/2018 22:43:27,public,0,22,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0,21993357


Let's engineer some additional event features:
- add event date as Year/Month/Day
- add day of week 
- add distance from nearest subway OR number of stations within X miles

Adding year/month/day and day of week columns.

In [181]:
# adding event date as Year/Month/Day
# let's first change the time to a datetime datatype
df_events['time'] = pd.to_datetime(df_events['time'])
df_events['time_m_d_y'] = df_events['time'].apply(lambda x: x.strftime('%Y-%m-%d'))     

In [182]:
# add column with day of week
df_events['time_m_d_y'] = pd.to_datetime(df_events['time_m_d_y'])
df_events['day_of_week'] = df_events['time_m_d_y'].dt.day_name()

In [183]:
df_events.head()

,created,description,duration_min,event_url,fee,headcount,id,maybe_rsvp_count,name,photo_url,...,visibility,waitlist_count,yes_rsvp_count,venue_neighborhood,venue_latlon,venue_country,venue_id,group_id,time_m_d_y,day_of_week
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...",NaN,...,public,0,42,Gramercy Park,"(40.723171, -73.997177)",United States,25315570.0,21993357,2018-09-04,Tuesday
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,NaN,...,public,0,64,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0,21993357,2018-09-06,Thursday
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,NaN,...,public,0,83,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0,21993357,2018-09-06,Thursday
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,252915161,0,How to get Hired as a Software Engineer,NaN,...,public,0,113,Gramercy Park,"(40.723171, -73.997177)",United States,25312065.0,21993357,2018-09-07,Friday
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254144933,0,Codesmith Online Info Session (Online Only),NaN,...,public,0,21,Gramercy Park,"(40.746731, -73.98967)",United States,25626092.0,21993357,2018-09-11,Tuesday


In [184]:
df_events['day_of_week'].value_counts()

Saturday     3221
Thursday     2192
Sunday       2137
Friday       1611
Wednesday    1464
Tuesday      1240
Monday       1001
Name: day_of_week, dtype: int64

Add distance from each venue all NYC subway stations.
Include a count of subways that are <= 0.25 miles from the venue.

In [185]:
# load subway station data
df_subway = pd.read_csv("NYC_Subway_Data.csv")

In [186]:
# dropping duplicate stations (file contains a location for each entry/exit point which is not what we need)
df_unique_subway = df_subway.drop_duplicates(subset=["Division", "Station Name"])

In [187]:
# preview the distinct subway station data
df_unique_subway

,Division,Line,Station Name,Station Latitude,Station Longitude,Route1,Route2,Route3,Route4,Route5,...,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.654490, -74.004499)"
5,BMT,4 Avenue,45th St,40.648939,-74.010006,R,NaN,NaN,NaN,NaN,...,False,NaN,True,4th Ave,45th St,NE,40.649389,-74.009333,"(40.648939, -74.010006)","(40.649389, -74.009333)"
9,BMT,4 Avenue,53rd St,40.645069,-74.014034,R,NaN,NaN,NaN,NaN,...,False,NaN,True,4th Ave,53rd St,SW,40.644653,-74.014690,"(40.645069, -74.014034)","(40.644653, -74.014690)"
14,BMT,4 Avenue,59th St,40.641362,-74.017881,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,59th St,NW,40.641606,-74.017897,"(40.641362, -74.017881)","(40.641606, -74.017897)"
20,BMT,4 Avenue,77th St,40.629742,-74.025510,R,NaN,NaN,NaN,NaN,...,False,NaN,True,4th Ave,77th St,NW,40.629550,-74.025731,"(40.629742, -74.02551)","(40.629550, -74.025731)"
23,BMT,4 Avenue,86th St,40.622687,-74.028398,R,NaN,NaN,NaN,NaN,...,False,NaN,True,4th Ave,86th St,SW,40.622656,-74.028547,"(40.622687, -74.028398)","(40.622656, -74.028547)"
26,BMT,4 Avenue,95th St,40.616622,-74.030876,R,NaN,NaN,NaN,NaN,...,False,NaN,True,4th Ave,95th St,SW,40.616021,-74.031383,"(40.616622, -74.030876)","(40.616021, -74.031383)"
31,BMT,4 Avenue,9th St,40.670847,-73.988302,F,G,R,NaN,NaN,...,False,NaN,True,4th Ave,9th St,NE,40.670387,-73.988480,"(40.670847, -73.988302)","(40.670387, -73.988480)"
33,BMT,4 Avenue,Atlantic Av-Barclays Ctr,40.683666,-73.978810,B,Q,D,N,R,...,True,NaN,True,4th Ave,Pacific St,NE,40.683805,-73.978487,"(40.683666, -73.97881)","(40.683805, -73.978487)"


In [188]:
# there are 425 unique stations
df_unique_subway.shape

(425, 32)

In [189]:
# convert the latitude and longitude into floats for distance calculation
df_unique_subway['Station Latitude'].astype(float)
df_unique_subway['Station Longitude'].astype(float)

0      -73.998091
2      -74.003549
5      -74.010006
9      -74.014034
14     -74.017881
20     -74.025510
23     -74.028398
26     -74.030876
31     -73.988302
33     -73.978810
34     -74.023377
37     -73.981824
43     -73.978810
45     -73.992872
48     -73.983110
52     -73.979189
59     -73.986229
60     -73.996209
70     -73.992821
78     -73.989938
82     -73.987823
92     -73.984569
101    -73.981329
118    -73.989779
120    -73.977450
128    -73.980305
136    -73.990862
142    -73.996204
147    -73.995048
152    -73.979678
          ...    
1748   -73.981848
1750   -74.005351
1754   -73.996353
1763   -73.986829
1767   -73.994791
1771   -73.995476
1774   -73.996895
1777   -73.998864
1783   -74.000610
1787   -73.994324
1788   -73.983765
1792   -73.993728
1800   -73.917757
1808   -73.887734
1812   -73.862633
1814   -73.860341
1817   -73.857473
1819   -73.854376
1821   -73.867352
1823   -73.868457
1826   -73.867164
1828   -73.873488
1830   -73.880049
1835   -73.891865
1841   -73

In [190]:
# create a new column with the converted latitude and longitutdes in a tuple
df_unique_subway['latlon'] = list(zip(df_unique_subway['Station Latitude'],df_unique_subway['Station Longitude']))

/Users/gracejeong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [191]:
# create a variable with a list of each station's (latitude, longitude)
subway_locations = list(df_unique_subway['latlon'])

In [192]:
def get_subway_distances(coord, subway_locations):
    """returns a list of distances from venue to each subway station in NYC, sorted from closest to farthest"""
    return sorted([haversine(coord, s, unit = 'mi') for s in subway_locations])

In [193]:
# import function created to get the distances of each venue to each subway station
# apply/lambda function to every event
df_events['subway_distances'] = df_events['venue_latlon'].apply(lambda x: get_subway_distances(x, subway_locations))

In [194]:
# create a column with a count of subway stations less than 0.5 miles from each venue
df_events['num_close_subways'] = df_events['subway_distances'].apply(lambda x: len([i for i in x if i <=0.5]))

In [197]:
# get a count of words in the description to see how descriptive the event is
df_events['descrip_tokens'] = df_events['description'].apply(lambda x: x.split(' '))

In [198]:
# preview the updated dataframe
df_events.head()

,created,description,duration_min,event_url,fee,headcount,id,maybe_rsvp_count,name,photo_url,...,venue_neighborhood,venue_latlon,venue_country,venue_id,group_id,time_m_d_y,day_of_week,subway_distances,num_close_subways,descrip_tokens
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...",NaN,...,Gramercy Park,"(40.723171, -73.997177)",United States,25315570.0,21993357,2018-09-04,Tuesday,"[0.06014082663839308, 0.08460125597085202, 0.1...",12,"[Join, us, in, person, or, tune, in, online!, ..."
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,NaN,...,Gramercy Park,"(40.74673, -73.98967)",United States,25626092.0,21993357,2018-09-06,Thursday,"[0.09959557289714283, 0.19629674920363338, 0.2...",9,"[Get, started, now, on, challenges, related, t..."
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,NaN,...,Gramercy Park,"(40.72317, -73.99718)",United States,25312065.0,21993357,2018-09-06,Thursday,"[0.06007690211738512, 0.08461572701356186, 0.1...",12,"[In, this, workshop, we’ll, get, a, clear, sen..."
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,252915161,0,How to get Hired as a Software Engineer,NaN,...,Gramercy Park,"(40.723171, -73.997177)",United States,25312065.0,21993357,2018-09-07,Friday,"[0.06014082663839308, 0.08460125597085202, 0.1...",12,"[The, number, of, opportunities, for, software..."
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254144933,0,Codesmith Online Info Session (Online Only),NaN,...,Gramercy Park,"(40.746731, -73.98967)",United States,25626092.0,21993357,2018-09-11,Tuesday,"[0.09965482405502167, 0.19628496446669605, 0.2...",9,"[Please, tune, into, the, stream, here:, https..."


In [494]:
df_events['time_hour'] = df_events.time.apply(lambda x: str(x)[-8:])

In [495]:
df_events['time_hour'].head()

0    22:30:00
1    00:00:00
2    22:30:00
3    22:30:00
4    01:30:00
Name: time_hour, dtype: object

In [492]:
df_events.time

0       2018-09-04 22:30:00
1       2018-09-06 00:00:00
2       2018-09-06 22:30:00
3       2018-09-07 22:30:00
4       2018-09-11 01:30:00
5       2018-09-11 22:30:00
6       2018-09-13 00:00:00
7       2018-09-13 22:30:00
8       2018-09-13 22:30:00
9       2018-09-15 16:00:00
10      2018-09-18 01:30:00
11      2018-09-20 00:00:00
12      2018-09-20 22:30:00
13      2018-09-21 22:30:00
14      2018-09-25 22:00:00
15      2018-09-25 22:30:00
16      2018-09-26 01:30:00
17      2018-09-27 00:00:00
18      2018-09-27 22:30:00
19      2018-09-27 23:00:00
20      2018-09-29 14:00:00
21      2018-09-11 13:00:00
22      2018-09-13 22:30:00
23      2018-09-27 22:30:00
24      2018-09-12 22:00:00
25      2018-09-19 22:00:00
26      2018-09-25 22:00:00
27      2018-09-26 22:30:00
28      2018-09-01 09:00:00
29      2018-09-01 10:00:00
                ...        
12836   2018-09-29 17:30:00
12837   2018-09-29 18:30:00
12838   2018-09-05 23:45:00
12839   2018-09-12 23:45:00
12840   2018-09-19 2

In [218]:
# function to remove special character tokens in the tokenzied descriptions

def remove_special_chars(some_list):
    remove = ["-", "--", "###", "##", "","•"]
    return [x for x in some_list if x not in remove]

In [219]:
# apply the remove_special_chars function to the descrip_tokens column
df_events['descrip_tokens'] = df_events['descrip_tokens'].apply(lambda x: remove_special_chars(x))

In [220]:
df_events['descrip_tokens'][2000]

['Please',
 'register',
 'for',
 'and',
 'tune',
 'into',
 'the',
 'stream',
 'here:',
 'https://zoom.us/j/544873117',
 'This',
 'week',
 'for',
 'Javascript',
 'the',
 'Hard',
 'Parts',
 'we',
 'cover',
 'Callbacks',
 'and',
 'Higher',
 'Order',
 'Functions.',
 'This',
 'includes',
 'how',
 'to',
 'handle',
 'or',
 'recreate',
 'the',
 'functions:',
 'map',
 'reduce',
 'forEach',
 'This',
 'is',
 'a',
 'concept',
 'that',
 '80%',
 'of',
 'JavaScript',
 'engineers',
 'do',
 'not',
 'understand',
 'but',
 'are',
 'at',
 'the',
 'core',
 'of',
 'every',
 'single',
 'successful',
 'application',
 'This',
 'is',
 'possible',
 'because',
 "we'll",
 'understand',
 'the',
 'essence',
 'of',
 'these',
 'tough',
 'concepts',
 'and',
 'then',
 'switch',
 'into',
 'pair-programming',
 'through',
 'the',
 'challenges',
 'to',
 'cement',
 'the',
 'concepts',
 'Schedule',
 '6:30',
 '7:00pm:',
 'Meet',
 'your',
 'future',
 'pair',
 'programming',
 'partner.',
 '7:00',
 '8:00pm:',
 'Core',
 'JavaScrip

In [221]:
# how add a column with a count of words in the descrip_tokens 
df_events['descrip_num_words'] = df_events['descrip_tokens'].apply(lambda x: len(x))

In [224]:
# create new column that notes whether there is a fee or no fee for the event
df_events['has_fee'] = df_events.fee.apply(lambda x: 0 if x == 0 else 1)

In [423]:
df_events.photo_url.astype(str)

0                                                      nan
1                                                      nan
2                                                      nan
3                                                      nan
4                                                      nan
5                                                      nan
6                                                      nan
7        https://secure.meetupstatic.com/photos/event/9...
8                                                      nan
9        https://secure.meetupstatic.com/photos/event/b...
10                                                     nan
11                                                     nan
12                                                     nan
13                                                     nan
14                                                     nan
15                                                     nan
16       https://secure.meetupstatic.com/photos/event/d.

In [447]:
# create new column that notes whether there was an event photo (1) or not (0)
df_events['has_event_photo'] = df_events.photo_url.apply(lambda x: 0 if str(x) == 'nan' else 1)

In [450]:
# drop the 'photo_url' column
df_events.drop(columns= ['photo_url'], inplace = True)

In [451]:
# preview the updated dataframe
df_events.head()

,created,description,duration_min,event_url,fee,headcount,id,maybe_rsvp_count,name,rating,...,venue_id,group_id,time_m_d_y,day_of_week,subway_distances,num_close_subways,descrip_tokens,descrip_num_words,has_fee,has_event_photo
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...","{'count': 0, 'average': 0}",...,25315570.0,21993357,2018-09-04,Tuesday,"[0.06014082663839308, 0.08460125597085202, 0.1...",12,"[Join, us, in, person, or, tune, in, online!, ...",137,0,0
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,"{'count': 0, 'average': 0}",...,25626092.0,21993357,2018-09-06,Thursday,"[0.09959557289714283, 0.19629674920363338, 0.2...",9,"[Get, started, now, on, challenges, related, t...",131,0,0
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,"{'count': 0, 'average': 0}",...,25312065.0,21993357,2018-09-06,Thursday,"[0.06007690211738512, 0.08461572701356186, 0.1...",12,"[In, this, workshop, we’ll, get, a, clear, sen...",204,0,0
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,252915161,0,How to get Hired as a Software Engineer,"{'count': 0, 'average': 0}",...,25312065.0,21993357,2018-09-07,Friday,"[0.06014082663839308, 0.08460125597085202, 0.1...",12,"[The, number, of, opportunities, for, software...",229,0,0
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254144933,0,Codesmith Online Info Session (Online Only),"{'count': 0, 'average': 0}",...,25626092.0,21993357,2018-09-11,Tuesday,"[0.09965482405502167, 0.19628496446669605, 0.2...",9,"[Please, tune, into, the, stream, here:, https...",165,0,0


In [452]:
# save cleaned dataframe into csv and json
df_events.to_csv("all_past_events_cleaned.csv")
df_events.to_json("all_past_events_cleaned.json")

In [453]:
df_events.columns

Index(['created', 'description', 'duration_min', 'event_url', 'fee',
       'headcount', 'id', 'maybe_rsvp_count', 'name', 'rating', 'rsvp_limit',
       'status', 'time', 'updated', 'visibility', 'waitlist_count',
       'yes_rsvp_count', 'venue_neighborhood', 'venue_latlon', 'venue_country',
       'venue_id', 'group_id', 'time_m_d_y', 'day_of_week', 'subway_distances',
       'num_close_subways', 'descrip_tokens', 'descrip_num_words', 'has_fee',
       'has_event_photo'],
      dtype='object')

In [479]:
df_num_past_events = pd.DataFrame(df_events.group_id.value_counts()).reset_index()

In [482]:
df_num_past_events.columns = ['group_id', 'num_past_events']
df_num_past_events.head()

,group_id,num_past_events
0,1414748,90
1,9513972,78
2,24480154,78
3,344877,76
4,9608102,75


In [454]:
len(df_events.group_id.unique())

2211

Left-merge the events and group dataframes on ```group_id```

In [455]:
df_events_w_groupcount = pd.merge(df_events, df_groups, how='left', on = 'group_id')

In [456]:
df_events_w_groupcount.head()

,created_x,description_x,duration_min,event_url,fee,headcount,id,maybe_rsvp_count,name_x,rating,...,name_y,state,status_y,urlname,visibility_y,who,category_name,organizer_id,yrs_since_created,created_date
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...","{'count': 0, 'average': 0}",...,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,"{'count': 0, 'average': 0}",...,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,"{'count': 0, 'average': 0}",...,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,252915161,0,How to get Hired as a Software Engineer,"{'count': 0, 'average': 0}",...,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254144933,0,Codesmith Online Info Session (Online Only),"{'count': 0, 'average': 0}",...,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02


Another left-merge to add number of past events (df_num_past_events).

In [483]:
df_events_w_groupcount = pd.merge(df_events, df_groups, how='left', on = 'group_id')
df_events_w_groupcount = pd.merge(df_events_w_groupcount, df_num_past_events, how= 'left', on = 'group_id')

In [488]:
df_events_w_groupcount.head()

,created_x,description_x,duration_min,event_url,fee,headcount,id,maybe_rsvp_count,name_x,rating,...,state,status_y,urlname,visibility_y,who,category_name,organizer_id,yrs_since_created,created_date,num_past_events
0,08/27/2018 17:36:07,Join us in person or tune in online! Livestrea...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254149786,0,"JavaScript The Easier Parts: Objects, Function...","{'count': 0, 'average': 0}",...,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02,21
1,08/27/2018 15:59:07,Get started now on challenges related to these...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254146381,0,Online Only JavaScript - The Hard Parts: Itera...,"{'count': 0, 'average': 0}",...,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02,21
2,08/27/2018 17:54:44,In this workshop we’ll get a clear sense of th...,150.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254150230,0,JavaScript The Hard Parts - Promises and Async...,"{'count': 0, 'average': 0}",...,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02,21
3,07/18/2018 21:06:34,The number of opportunities for software engin...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,252915161,0,How to get Hired as a Software Engineer,"{'count': 0, 'average': 0}",...,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02,21
4,08/27/2018 15:24:18,Please tune into the stream here: https://zoom...,120.0,https://www.meetup.com/Build-with-Code-New-Yor...,0.0,0,254144933,0,Codesmith Online Info Session (Online Only),"{'count': 0, 'average': 0}",...,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02,21


Rename column headers for clarify (event vs. group info).

In [489]:
df_events_w_groupcount.columns

Index(['created_x', 'description_x', 'duration_min', 'event_url', 'fee',
       'headcount', 'id', 'maybe_rsvp_count', 'name_x', 'rating', 'rsvp_limit',
       'status_x', 'time', 'updated', 'visibility_x', 'waitlist_count',
       'yes_rsvp_count', 'venue_neighborhood', 'venue_latlon', 'venue_country',
       'venue_id', 'group_id', 'time_m_d_y', 'day_of_week', 'subway_distances',
       'num_close_subways', 'descrip_tokens', 'descrip_num_words', 'has_fee',
       'has_event_photo', 'created_y', 'description_y', 'join_mode', 'lat',
       'link', 'localized_country_name', 'localized_location', 'lon',
       'members', 'name_y', 'state', 'status_y', 'urlname', 'visibility_y',
       'who', 'category_name', 'organizer_id', 'yrs_since_created',
       'created_date', 'num_past_events'],
      dtype='object')

In [490]:
df_events_w_groupcount.rename(columns = {'created_x': 'event_created',
                                         'description_x': 'event_description',
                                         'duration_min': 'event_duration',
                                         'headcount': 'event_headcount',
                                         'id': 'event_id',
                                         'name_x': 'event_name',
                                         'rating': 'event_rating',
                                         'status_x': 'event_status',
                                         'time': 'event_time',
                                         'updated': 'event_updated',
                                         'visibility_x': 'event_visibility',
                                         'descrip_tokens': 'event_descrip_tokens',
                                         'descrip_num_words':'event_descrip_num_words',
                                         'has_fee': 'has_event_fee',
                                         'created_y': 'group_created',
                                         'description_y': 'group_description',
                                         'join_mode': 'group_join_mode',
                                         'lat': 'group_lat',
                                         'lon': 'group_lon',
                                         'link': 'group_link',
                                         'state': 'group_state',
                                         'members': 'num_members',
                                         'name_y': 'group_name',
                                         'status_y': 'group_status',
                                         'urlname': 'group_urlname',
                                         'visibility_y': 'group_visibility',
                                         'who': 'group_who',
                                         'category_name': 'group_category',
                                         'organizer_id': 'group_organizer_id',
                                         'yrs_since_created': 'group_yrs_est',
                                         'created_date':'group_created_date'
                                        }, inplace =True)

In [491]:
# save the merged dataframe
df_events_w_groupcount.to_pickle('df_events_w_group.pickle')

***
<a id='groups'></a>
### 2. Meetup Groups

#### Load Data

In [302]:
# open all_groups file
with open('all_groups.pkl', 'rb') as f:
    all_groups = pickle.load(f)

In [303]:
# convert to dataframe
df_groups = pd.DataFrame(all_groups)

In [304]:
df_groups.shape

(8632, 29)

In [305]:
df_groups.head()

,category,city,country,created,description,group_photo,id,is_pro_hidden,join_mode,key_photo,...,organizer,pro_network,score,state,status,timezone,untranslated_city,urlname,visibility,who
0,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",New York,US,1484876702000,<p>Build with Code hosts free weekly JavaScrip...,NaN,21993357,NaN,open,"{'id': 464860413, 'highres_link': 'https://sec...",...,"{'id': 218119162, 'name': 'Jenny Mith', 'bio':...",NaN,1.0,NY,active,US/Eastern,New York,Build-with-Code-New-York,public,Engineers
1,"{'id': 2, 'name': 'Career & Business', 'shortn...",New York,US,1550615516000,<p>The TechDay New York team invites you to jo...,NaN,31207091,NaN,open,"{'id': 480306005, 'highres_link': 'https://sec...",...,"{'id': 263284450, 'name': 'Ana ', 'bio': '', '...",NaN,1.0,NY,active,US/Eastern,New York,TechDayHQ,public,Members
2,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",New York,US,1047953152000,<p>The NYC NoSQL &amp; NewSQL Group <br> (form...,"{'id': 460182357, 'highres_link': 'https://sec...",107592,NaN,open,"{'id': 466506912, 'highres_link': 'https://sec...",...,"{'id': 6618661, 'name': 'Eric David Benari', '...",NaN,1.0,NY,active,US/Eastern,New York,mysqlnyc,public,Data Enthusiasts
3,"{'id': 23, 'name': 'Outdoors & Adventure', 'sh...",New York,US,1548684384000,<p><span>The Awesome Events Meetup Group is th...,NaN,31031999,NaN,open,"{'id': 480057227, 'highres_link': 'https://sec...",...,"{'id': 236287112, 'name': 'Justin', 'bio': '',...",NaN,1.0,NY,active,US/Eastern,New York,awesome-events,public,Awesome People
4,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",New York,US,1321563802000,<p><span>Data Driven NYC (organized by FirstMa...,"{'id': 442920809, 'highres_link': 'https://sec...",2829432,NaN,approval,"{'id': 442991280, 'highres_link': 'https://sec...",...,"{'id': 2369792, 'name': 'Matt Turck', 'bio': '...",NaN,1.0,NY,active,US/Eastern,New York,DataDrivenNYC,public,Members


In [306]:
# rename id to group_id
df_groups.rename(columns ={'id':'group_id'}, inplace = True)

In [307]:
df_groups.isna().sum()

category                     8
city                         0
country                      0
created                      0
description                  0
group_photo               4533
group_id                     0
is_pro_hidden             8628
join_mode                    0
key_photo                 1484
lat                          0
link                         0
localized_country_name       0
localized_location           0
lon                          0
members                      0
meta_category              154
name                         0
next_event                5852
organizer                    0
pro_network               8318
score                        0
state                        0
status                       0
timezone                     0
untranslated_city            0
urlname                      0
visibility                   0
who                          0
dtype: int64

We'll deal with most of the missing values in this dataset by dropping columns we won't need:
- ```is_pro_hidden```, ```pro_network```, ```next_event```, ```key_photo```, ```group_photo```, ```timezone```, ```untranslated_city```, ```score```, ```country```, ```city```, ```meta_category``` (contains the same info as ```category```)


In [308]:
df_groups.drop(columns = ['is_pro_hidden', 'pro_network', 'next_event', 'key_photo', 'group_photo',
                         'timezone', 'untranslated_city', 'score', 'country', 'city', 'meta_category'], 
               inplace = True)

In [309]:
df_groups.head()

,category,created,description,group_id,join_mode,lat,link,localized_country_name,localized_location,lon,members,name,organizer,state,status,urlname,visibility,who
0,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1484876702000,<p>Build with Code hosts free weekly JavaScrip...,21993357,open,40.75,https://www.meetup.com/Build-with-Code-New-York/,USA,"New York, NY",-73.99,8050,Build with Code - New York City,"{'id': 218119162, 'name': 'Jenny Mith', 'bio':...",NY,active,Build-with-Code-New-York,public,Engineers
1,"{'id': 2, 'name': 'Career & Business', 'shortn...",1550615516000,<p>The TechDay New York team invites you to jo...,31207091,open,40.75,https://www.meetup.com/TechDayHQ/,USA,"New York, NY",-73.99,1361,TechDay Meetup,"{'id': 263284450, 'name': 'Ana ', 'bio': '', '...",NY,active,TechDayHQ,public,Members
2,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1047953152000,<p>The NYC NoSQL &amp; NewSQL Group <br> (form...,107592,open,40.75,https://www.meetup.com/mysqlnyc/,USA,"New York, NY",-73.99,24226,"🔥 SQL NYC, The NoSQL & NewSQL Database Big Dat...","{'id': 6618661, 'name': 'Eric David Benari', '...",NY,active,mysqlnyc,public,Data Enthusiasts
3,"{'id': 23, 'name': 'Outdoors & Adventure', 'sh...",1548684384000,<p><span>The Awesome Events Meetup Group is th...,31031999,open,40.78,https://www.meetup.com/awesome-events/,USA,"New York, NY",-73.96,1694,Awesome Events,"{'id': 236287112, 'name': 'Justin', 'bio': '',...",NY,active,awesome-events,public,Awesome People
4,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1321563802000,<p><span>Data Driven NYC (organized by FirstMa...,2829432,approval,40.76,https://www.meetup.com/DataDrivenNYC/,USA,"New York, NY",-73.97,17382,Data Driven NYC (a FirstMark Event),"{'id': 2369792, 'name': 'Matt Turck', 'bio': '...",NY,active,DataDrivenNYC,public,Members


In [310]:
# clean text in descriptions
df_groups.description = df_groups.description.apply(lambda x: clean_text(x))

In [311]:
df_groups.head()

,category,created,description,group_id,join_mode,lat,link,localized_country_name,localized_location,lon,members,name,organizer,state,status,urlname,visibility,who
0,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1484876702000,Build with Code hosts free weekly JavaScript a...,21993357,open,40.75,https://www.meetup.com/Build-with-Code-New-York/,USA,"New York, NY",-73.99,8050,Build with Code - New York City,"{'id': 218119162, 'name': 'Jenny Mith', 'bio':...",NY,active,Build-with-Code-New-York,public,Engineers
1,"{'id': 2, 'name': 'Career & Business', 'shortn...",1550615516000,The TechDay New York team invites you to join ...,31207091,open,40.75,https://www.meetup.com/TechDayHQ/,USA,"New York, NY",-73.99,1361,TechDay Meetup,"{'id': 263284450, 'name': 'Ana ', 'bio': '', '...",NY,active,TechDayHQ,public,Members
2,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1047953152000,The NYC NoSQL NewSQL Group (formerly known a...,107592,open,40.75,https://www.meetup.com/mysqlnyc/,USA,"New York, NY",-73.99,24226,"🔥 SQL NYC, The NoSQL & NewSQL Database Big Dat...","{'id': 6618661, 'name': 'Eric David Benari', '...",NY,active,mysqlnyc,public,Data Enthusiasts
3,"{'id': 23, 'name': 'Outdoors & Adventure', 'sh...",1548684384000,The Awesome Events Meetup Group is the real-li...,31031999,open,40.78,https://www.meetup.com/awesome-events/,USA,"New York, NY",-73.96,1694,Awesome Events,"{'id': 236287112, 'name': 'Justin', 'bio': '',...",NY,active,awesome-events,public,Awesome People
4,"{'id': 34, 'name': 'Tech', 'shortname': 'tech'...",1321563802000,"Data Driven NYC (organized by FirstMark), is a...",2829432,approval,40.76,https://www.meetup.com/DataDrivenNYC/,USA,"New York, NY",-73.97,17382,Data Driven NYC (a FirstMark Event),"{'id': 2369792, 'name': 'Matt Turck', 'bio': '...",NY,active,DataDrivenNYC,public,Members


Let's look at the ```category``` column in more detail and extract just the information we want in the main ```df_groups``` dataframe.

In [312]:
df_category = df_groups['category'].apply(pd.Series)
df_category.head()

,id,name,shortname,sort_name,0
0,34.0,Tech,tech,Tech,NaN
1,2.0,Career & Business,career-business,Career & Business,NaN
2,34.0,Tech,tech,Tech,NaN
3,23.0,Outdoors & Adventure,outdoors-adventure,Outdoors & Adventure,NaN
4,34.0,Tech,tech,Tech,NaN


In [313]:
df_category.isna().sum()/(len(df_category))*100

id             0.092678
name           0.092678
shortname      0.092678
sort_name      0.092678
0            100.000000
dtype: float64

In [314]:
# replace NaNs
df_category['shortname'].fillna('None',inplace=True)

In [315]:
# add columns to main dataframe and drop 'category'
df_groups['category_name'] = df_category['shortname']
df_groups.drop(columns = ['category'], inplace=True)

In [316]:
df_groups.head()

,created,description,group_id,join_mode,lat,link,localized_country_name,localized_location,lon,members,name,organizer,state,status,urlname,visibility,who,category_name
0,1484876702000,Build with Code hosts free weekly JavaScript a...,21993357,open,40.75,https://www.meetup.com/Build-with-Code-New-York/,USA,"New York, NY",-73.99,8050,Build with Code - New York City,"{'id': 218119162, 'name': 'Jenny Mith', 'bio':...",NY,active,Build-with-Code-New-York,public,Engineers,tech
1,1550615516000,The TechDay New York team invites you to join ...,31207091,open,40.75,https://www.meetup.com/TechDayHQ/,USA,"New York, NY",-73.99,1361,TechDay Meetup,"{'id': 263284450, 'name': 'Ana ', 'bio': '', '...",NY,active,TechDayHQ,public,Members,career-business
2,1047953152000,The NYC NoSQL NewSQL Group (formerly known a...,107592,open,40.75,https://www.meetup.com/mysqlnyc/,USA,"New York, NY",-73.99,24226,"🔥 SQL NYC, The NoSQL & NewSQL Database Big Dat...","{'id': 6618661, 'name': 'Eric David Benari', '...",NY,active,mysqlnyc,public,Data Enthusiasts,tech
3,1548684384000,The Awesome Events Meetup Group is the real-li...,31031999,open,40.78,https://www.meetup.com/awesome-events/,USA,"New York, NY",-73.96,1694,Awesome Events,"{'id': 236287112, 'name': 'Justin', 'bio': '',...",NY,active,awesome-events,public,Awesome People,outdoors-adventure
4,1321563802000,"Data Driven NYC (organized by FirstMark), is a...",2829432,approval,40.76,https://www.meetup.com/DataDrivenNYC/,USA,"New York, NY",-73.97,17382,Data Driven NYC (a FirstMark Event),"{'id': 2369792, 'name': 'Matt Turck', 'bio': '...",NY,active,DataDrivenNYC,public,Members,tech


Let's look at the ```organizer``` column in more detail and extract just the information we want in the main ```df_groups``` dataframe.

In [317]:
df_org = df_groups['organizer'].apply(pd.Series)
df_org.head()

,id,name,bio,photo
0,218119162,Jenny Mith,,"{'id': 262996470, 'highres_link': 'https://sec..."
1,263284450,Ana,,"{'id': 281661741, 'highres_link': 'https://sec..."
2,6618661,Eric David Benari,,"{'id': 4946659, 'highres_link': 'https://secur..."
3,236287112,Justin,,"{'id': 284561488, 'highres_link': 'https://sec..."
4,2369792,Matt Turck,"Managing Director, FirstMark Capital","{'id': 266918773, 'highres_link': 'https://sec..."


In [318]:
df_org.isna().sum()/(len(df_org))*100

id       0.000000
name     0.000000
bio      0.000000
photo    6.452734
dtype: float64

In [319]:
# let's keep just the organizer's id just in case and drop the 'organizer' column from df_groups
df_groups['organizer_id'] = df_org['id']
df_groups.drop(columns = ['organizer'], inplace =True)

In [320]:
df_groups.head()

,created,description,group_id,join_mode,lat,link,localized_country_name,localized_location,lon,members,name,state,status,urlname,visibility,who,category_name,organizer_id
0,1484876702000,Build with Code hosts free weekly JavaScript a...,21993357,open,40.75,https://www.meetup.com/Build-with-Code-New-York/,USA,"New York, NY",-73.99,8050,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162
1,1550615516000,The TechDay New York team invites you to join ...,31207091,open,40.75,https://www.meetup.com/TechDayHQ/,USA,"New York, NY",-73.99,1361,TechDay Meetup,NY,active,TechDayHQ,public,Members,career-business,263284450
2,1047953152000,The NYC NoSQL NewSQL Group (formerly known a...,107592,open,40.75,https://www.meetup.com/mysqlnyc/,USA,"New York, NY",-73.99,24226,"🔥 SQL NYC, The NoSQL & NewSQL Database Big Dat...",NY,active,mysqlnyc,public,Data Enthusiasts,tech,6618661
3,1548684384000,The Awesome Events Meetup Group is the real-li...,31031999,open,40.78,https://www.meetup.com/awesome-events/,USA,"New York, NY",-73.96,1694,Awesome Events,NY,active,awesome-events,public,Awesome People,outdoors-adventure,236287112
4,1321563802000,"Data Driven NYC (organized by FirstMark), is a...",2829432,approval,40.76,https://www.meetup.com/DataDrivenNYC/,USA,"New York, NY",-73.97,17382,Data Driven NYC (a FirstMark Event),NY,active,DataDrivenNYC,public,Members,tech,2369792


In [321]:
df_groups.columns

Index(['created', 'description', 'group_id', 'join_mode', 'lat', 'link',
       'localized_country_name', 'localized_location', 'lon', 'members',
       'name', 'state', 'status', 'urlname', 'visibility', 'who',
       'category_name', 'organizer_id'],
      dtype='object')

In [322]:
# let's get a count to see how long each group has been around by subtracting 'created' timestamp from May 1st.
df_groups['yrs_since_created'] = ((1556683200000 - df_groups['created'])/86400000)/365

In [323]:
df_groups['created_date'] = df_groups['created'].apply(lambda x:time.strftime('%m/%d/%Y %H:%M:%S', time.gmtime(x/1000.)))

In [324]:
df_groups.head()

,created,description,group_id,join_mode,lat,link,localized_country_name,localized_location,lon,members,name,state,status,urlname,visibility,who,category_name,organizer_id,yrs_since_created,created_date
0,1484876702000,Build with Code hosts free weekly JavaScript a...,21993357,open,40.75,https://www.meetup.com/Build-with-Code-New-York/,USA,"New York, NY",-73.99,8050,Build with Code - New York City,NY,active,Build-with-Code-New-York,public,Engineers,tech,218119162,2.276969,01/20/2017 01:45:02
1,1550615516000,The TechDay New York team invites you to join ...,31207091,open,40.75,https://www.meetup.com/TechDayHQ/,USA,"New York, NY",-73.99,1361,TechDay Meetup,NY,active,TechDayHQ,public,Members,career-business,263284450,0.192405,02/19/2019 22:31:56
2,1047953152000,The NYC NoSQL NewSQL Group (formerly known a...,107592,open,40.75,https://www.meetup.com/mysqlnyc/,USA,"New York, NY",-73.99,24226,"🔥 SQL NYC, The NoSQL & NewSQL Database Big Dat...",NY,active,mysqlnyc,public,Data Enthusiasts,tech,6618661,16.131724,03/18/2003 02:05:52
3,1548684384000,The Awesome Events Meetup Group is the real-li...,31031999,open,40.78,https://www.meetup.com/awesome-events/,USA,"New York, NY",-73.96,1694,Awesome Events,NY,active,awesome-events,public,Awesome People,outdoors-adventure,236287112,0.253641,01/28/2019 14:06:24
4,1321563802000,"Data Driven NYC (organized by FirstMark), is a...",2829432,approval,40.76,https://www.meetup.com/DataDrivenNYC/,USA,"New York, NY",-73.97,17382,Data Driven NYC (a FirstMark Event),NY,active,DataDrivenNYC,public,Members,tech,2369792,7.455587,11/17/2011 21:03:22


In [325]:
df_groups.to_csv('all_groups_cleaned.csv')

In [326]:
df_groups.to_json('all_groups_cleaned.json')

***
<a id='members'></a>
### 3. Meetup Members

Here we will merge two dataframes containing information on members. The first is information scraped from member profile pages and the other is member info obtained from the members API endpoint.


#### Scraped data

In [351]:
# importing member profiles scraped:
with open('member_profiles_16000.pkl', 'rb') as f:
    member_profiles = pickle.load(f)

In [352]:
print(f"Scraped {len(member_profiles)} profiles")

Scraped 15990 profiles


In [353]:
# view data in dataframe
df_members = pd.DataFrame(member_profiles)
df_members.head()

,groups,interests,member_url
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863


In [354]:
# count number of items in groups and interest; will drop members without any group or interest information
df_members['num_groups'] = df_members.groups.apply(lambda x: len(x))
df_members['num_interests'] = df_members.interests.apply(lambda x: len(x))

In [355]:
# get the indices of rows that are missing both group and interest data; use indices to drop rows
missing_groups_ints = df_members[(df_members['num_groups'] == 0) & (df_members['num_interests']==0)]
df_members.drop(index = missing_groups_ints.index, axis = 0, inplace = True)

In [356]:
# now's lets also drop members missing either groups or interests (1,105 in total) so that we only work with 
# users with full info
missing_groups = df_members[df_members['num_groups'] == 0]
missing_ints = df_members[df_members['num_interests'] == 0]

df_members.drop(index = missing_groups.index, axis = 0, inplace = True)
df_members.drop(index = missing_ints.index, axis = 0, inplace = True)

In [357]:
df_members.shape

(14879, 5)

In [358]:
# preview the updated dataframe
df_members.head()

,groups,interests,member_url,num_groups,num_interests
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912,7,4
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603,8,23
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602,3,9
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532,12,51
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863,9,14


In [360]:
# save final dataframe to json and pickle
df_members.to_json("member_profiles_1600_cleaned.json")
df_members.to_pickle("df_scraped_profiles_cleaned.pickle")

#### API data

In [361]:
# getting back pickled dataframe containing the API member info
df_membersapi = pd.read_pickle('df_unique_members.pickle')

In [362]:
df_membersapi.shape

(234609, 17)

In [363]:
df_membersapi.head()

,bio,city,country,hometown,id,joined,lat,link,lon,name,other_services,photo,self,state,status,topics,visited
0,NaN,Bronx,us,NaN,276413419,1.552398e+12,40.82,http://www.meetup.com/members/276413419,-73.92,Charisse,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'newtech', 'name': 'New Technology...",1.552398e+12
1,NaN,New York,us,NaN,245744462,1.515612e+12,40.75,http://www.meetup.com/members/245744462,-73.99,Ibrahima Diallo,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'newtech', 'name': 'New Technology...",1.515612e+12
2,NaN,New York,us,NaN,273936256,1.549559e+12,40.75,http://www.meetup.com/members/273936256,-73.99,Victoria Read,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,[],1.549559e+12
3,NaN,New York,us,NaN,258398074,1.531030e+12,40.75,http://www.meetup.com/members/258398074,-73.99,+V信feng4343注册得99链接186053.com,{},NaN,{'common': {}},NY,active,[],1.531030e+12
4,NaN,New York,us,NaN,259737701,1.552287e+12,40.75,http://www.meetup.com/members/259737701,-73.99,¥en,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,[],1.552287e+12


In [364]:
# renaming the link column to stage for merging with scraped dataframe
df_membersapi.rename(columns={'link':'member_url'}, inplace = True)
df_membersapi.head()

,bio,city,country,hometown,id,joined,lat,member_url,lon,name,other_services,photo,self,state,status,topics,visited
0,NaN,Bronx,us,NaN,276413419,1.552398e+12,40.82,http://www.meetup.com/members/276413419,-73.92,Charisse,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'newtech', 'name': 'New Technology...",1.552398e+12
1,NaN,New York,us,NaN,245744462,1.515612e+12,40.75,http://www.meetup.com/members/245744462,-73.99,Ibrahima Diallo,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'newtech', 'name': 'New Technology...",1.515612e+12
2,NaN,New York,us,NaN,273936256,1.549559e+12,40.75,http://www.meetup.com/members/273936256,-73.99,Victoria Read,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,[],1.549559e+12
3,NaN,New York,us,NaN,258398074,1.531030e+12,40.75,http://www.meetup.com/members/258398074,-73.99,+V信feng4343注册得99链接186053.com,{},NaN,{'common': {}},NY,active,[],1.531030e+12
4,NaN,New York,us,NaN,259737701,1.552287e+12,40.75,http://www.meetup.com/members/259737701,-73.99,¥en,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,[],1.552287e+12


#### Merged data

In [365]:
# left merge of dataframe on member_url column
full_df_members = pd.merge(df_members, df_membersapi, how = 'left', on= 'member_url')

In [366]:
full_df_members.shape

(14879, 21)

In [367]:
# preview the merged dataframe
full_df_members.head()

,groups,interests,member_url,num_groups,num_interests,bio,city,country,hometown,id,...,lat,lon,name,other_services,photo,self,state,status,topics,visited
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912,7,4,NaN,Secaucus,us,secaucus,57678912,...,40.79,-74.06,Dee,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NJ,active,"[{'urlkey': 'business-referral-networking', 'n...",1.466428e+12
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603,8,23,NaN,New York,us,"St. Gallen, Switzerland",230923603,...,40.72,-73.98,Alistair Barrell,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'foodie', 'name': 'Foodie', 'id': ...",1.554763e+12
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602,3,9,NaN,New York,us,NaN,24427602,...,40.72,-74.00,Beth Barber,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'visual-studio', 'name': 'Visual S...",1.447760e+12
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532,12,51,NaN,New Haven,us,New Haven,75979532,...,41.33,-72.97,Kathy,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},CT,active,"[{'urlkey': 'coffee', 'name': 'Coffee', 'id': ...",1.514860e+12
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863,9,14,NaN,West Hempstead,us,NaN,279891863,...,40.69,-73.65,Karen White Kelly,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'hiphop', 'name': 'Hip Hop', 'id':...",1.556335e+12


In [368]:
full_df_members.columns

Index(['groups', 'interests', 'member_url', 'num_groups', 'num_interests',
       'bio', 'city', 'country', 'hometown', 'id', 'joined', 'lat', 'lon',
       'name', 'other_services', 'photo', 'self', 'state', 'status', 'topics',
       'visited'],
      dtype='object')

In [370]:
# we can drop the self columns since they are all empty
full_df_members.self.value_counts()

{'common': {}}    14879
Name: self, dtype: int64

In [ ]:
full_df_members.drop(columns = ['self'], inplace = True)

In [371]:
# the other_services column contains other social media contacts for the member
full_df_members.other_services.value_counts()

{}                                                                                                                                                                                                                                                                                               13778
{'twitter': {'identifier': 'http://'}}                                                                                                                                                                                                                                                               3
{'twitter': {'identifier': '@redvioletdar'}}                                                                                                                                                                                                                                                         1
{'twitter': {'identifier': '@HarlemFund'}, 'linkedin': {'identifier': 'http://www.linkedin.com/in/thomas-lopez-pier

In [380]:
# create a column with a count of the number of connected social media accounts
full_df_members['num_sm_accounts'] = full_df_members.other_services.apply(lambda x: len(x))

In [381]:
full_df_members.head()

,groups,interests,member_url,num_groups,num_interests,bio,city,country,hometown,id,...,lon,name,other_services,photo,self,state,status,topics,visited,num_sm_accounts
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912,7,4,NaN,Secaucus,us,secaucus,57678912,...,-74.06,Dee,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NJ,active,"[{'urlkey': 'business-referral-networking', 'n...",1.466428e+12,0
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603,8,23,NaN,New York,us,"St. Gallen, Switzerland",230923603,...,-73.98,Alistair Barrell,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'foodie', 'name': 'Foodie', 'id': ...",1.554763e+12,0
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602,3,9,NaN,New York,us,NaN,24427602,...,-74.00,Beth Barber,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'visual-studio', 'name': 'Visual S...",1.447760e+12,0
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532,12,51,NaN,New Haven,us,New Haven,75979532,...,-72.97,Kathy,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},CT,active,"[{'urlkey': 'coffee', 'name': 'Coffee', 'id': ...",1.514860e+12,0
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863,9,14,NaN,West Hempstead,us,NaN,279891863,...,-73.65,Karen White Kelly,{},{'highres_link': 'https://secure.meetupstatic....,{'common': {}},NY,active,"[{'urlkey': 'hiphop', 'name': 'Hip Hop', 'id':...",1.556335e+12,0


In [382]:
# drop the other_services column
full_df_members.drop(columns = ['other_services'], inplace =True)

In [396]:
(full_df_members.isna().sum()/len(full_df_members))*100

groups             0.0
interests          0.0
member_url         0.0
num_groups         0.0
num_interests      0.0
bio                0.0
city               0.0
country            0.0
hometown           0.0
id                 0.0
joined             0.0
lat                0.0
lon                0.0
name               0.0
self               0.0
state              0.0
status             0.0
topics             0.0
visited            0.0
num_sm_accounts    0.0
has_photo          0.0
dtype: float64

In [395]:
# fill in the state, bio, and hometown NaN values with 'None'
full_df_members.state.fillna('None', inplace = True)
full_df_members.bio.fillna('None', inplace = True)
full_df_members.hometown.fillna('None', inplace = True)

In [390]:
# create a new column indicating whether member has a photo (1) or not (0) to replace the 'photo' column
full_df_members['has_photo'] = full_df_members.photo.apply(lambda x: 0 if x == None else 1)

In [392]:
# drop the 'photo' column
full_df_members.drop(columns = ['photo'], inplace = True)

In [393]:
full_df_members.head()

,groups,interests,member_url,num_groups,num_interests,bio,city,country,hometown,id,...,lat,lon,name,self,state,status,topics,visited,num_sm_accounts,has_photo
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912,7,4,NaN,Secaucus,us,secaucus,57678912,...,40.79,-74.06,Dee,{'common': {}},NJ,active,"[{'urlkey': 'business-referral-networking', 'n...",1.466428e+12,0,1
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603,8,23,NaN,New York,us,"St. Gallen, Switzerland",230923603,...,40.72,-73.98,Alistair Barrell,{'common': {}},NY,active,"[{'urlkey': 'foodie', 'name': 'Foodie', 'id': ...",1.554763e+12,0,1
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602,3,9,NaN,New York,us,NaN,24427602,...,40.72,-74.00,Beth Barber,{'common': {}},NY,active,"[{'urlkey': 'visual-studio', 'name': 'Visual S...",1.447760e+12,0,1
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532,12,51,NaN,New Haven,us,New Haven,75979532,...,41.33,-72.97,Kathy,{'common': {}},CT,active,"[{'urlkey': 'coffee', 'name': 'Coffee', 'id': ...",1.514860e+12,0,1
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863,9,14,NaN,West Hempstead,us,NaN,279891863,...,40.69,-73.65,Karen White Kelly,{'common': {}},NY,active,"[{'urlkey': 'hiphop', 'name': 'Hip Hop', 'id':...",1.556335e+12,0,1


In [405]:
# can drop 'topic' column as it contains the same info as 'interests'
full_df_members.drop(columns= ['topics'], inplace = True)

In [406]:
full_df_members.head()

,groups,interests,member_url,num_groups,num_interests,bio,city,country,hometown,id,joined,lat,lon,name,self,state,status,visited,num_sm_accounts,has_photo
0,"[Closing Deals in 6 Inch Heels NYC, Entreprene...","[Professional Development, Professional Women,...",http://www.meetup.com/members/57678912,7,4,None,Secaucus,us,secaucus,57678912,1.459463e+12,40.79,-74.06,Dee,{'common': {}},NJ,active,1.466428e+12,0,1
1,"[Ann Arbor Web Accessibility, Data Driven NYC ...","[Adventure, Language & Culture, Nightlife, Bac...",http://www.meetup.com/members/230923603,8,23,None,New York,us,"St. Gallen, Switzerland",230923603,1.537537e+12,40.72,-73.98,Alistair Barrell,{'common': {}},NY,active,1.554763e+12,0,1
2,"[ArtForward, Central Park Sketching & Art Meet...","[Theater, Performing Arts, Walking, Writing, A...",http://www.meetup.com/members/24427602,3,9,None,New York,us,None,24427602,1.436840e+12,40.72,-74.00,Beth Barber,{'common': {}},NY,active,1.447760e+12,0,1
3,"[#Resist: Danbury, Adult Day Camp, Black Nonbe...","[Museum, Cooking Dinner Parties, Wine, Healthy...",http://www.meetup.com/members/75979532,12,51,None,New Haven,us,New Haven,75979532,1.468890e+12,41.33,-72.97,Kathy,{'common': {}},CT,active,1.514860e+12,0,1
4,['NYC- Small Business and Entrepreneurs Networ...,"[Hip Hop, Wine, Business Strategy, Dining Out,...",http://www.meetup.com/members/279891863,9,14,None,West Hempstead,us,None,279891863,1.556335e+12,40.69,-73.65,Karen White Kelly,{'common': {}},NY,active,1.556335e+12,0,1


In [407]:
# save the cleaned dataframe
full_df_members.to_pickle("full_df_members.pickle")